In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [4]:
predictors = pd.read_excel('./predictors_monthly.xlsx')
commodity = pd.read_excel('./Commodities_Monthly.xlsx')
futures = pd.read_excel('./Futures_Monthly.xlsx')

In [5]:
predictors = predictors[46:].reset_index(drop=True)
predictors.set_index('date', inplace=True)
def preprocess(commodity, drop_col):
    commodity = commodity[35:].reset_index(drop=True)
    commodity['date'] = predictors.index
    commodity = commodity.drop(['Year_Month', drop_col], axis=1)
    commodity.set_index('date', inplace=True)
    return commodity
commodity = preprocess(commodity, 'Grains_Spot')
futures = preprocess(futures, 'Grains')

In [6]:
predictors = predictors[predictors.columns[:14]]

In [7]:
predictors['CPI Change'] = predictors['CPURNSA Index'].diff()
predictors = predictors[1:]
commodity = commodity[1:]
futures = futures[1:]
pred_orig = predictors
comm_orig = commodity
fut_orig = futures
# Predict 3 months into the future
predictors = predictors[:-3]
futures = futures[:-3]
commodity = commodity[3:]
# Normalize Predictors
predictors = (predictors-predictors.min())/(predictors.max()-predictors.min())
predictors = predictors.drop(labels='MSCI US Equity', axis=1)

In [16]:
commodity_lag = commodity.shift(3)

In [17]:
commodity_lag

,Agriculture_Spot,Energy_Spot,Industrial_Metals_Spot,Livestock_Spot,Precious_Metals_Spot
date,,,,,
2005-03-01,NaN,NaN,NaN,NaN,NaN
2005-04-01,NaN,NaN,NaN,NaN,NaN
2005-05-01,NaN,NaN,NaN,NaN,NaN
2005-06-01,149.016582,322.704968,133.276273,140.862305,131.145723
2005-07-01,145.138614,324.381038,129.501729,139.126286,129.327733
...,...,...,...,...,...
2022-04-01,418.312100,406.502375,313.109270,202.670920,528.257555
2022-05-01,448.131305,447.854963,325.604416,219.777547,539.362626
2022-06-01,496.890904,527.450922,380.813709,220.300878,571.097565


In [19]:
commodity_lag.to_csv("baseline.csv")

In [28]:
results = []
for col in commodity.columns:
    rmse = mean_squared_error(commodity[-24:][col], commodity_lag[-24:][col], squared=False)
    mae = mean_absolute_error(commodity[-24:][col], commodity_lag[-24:][col])
    r2 = r2_score(commodity[-24:][col], commodity_lag[-24:][col])
    results.append({"Commodity":col, "MAE":mae, "RMSE":rmse, "R2": r2})
pd.DataFrame(results)
        
    

,Commodity,MAE,RMSE,R2
0,Agriculture_Spot,43.461875,50.694817,0.391676
1,Energy_Spot,58.678934,75.616515,0.654465
2,Industrial_Metals_Spot,30.943094,40.579889,0.136295
3,Livestock_Spot,14.553733,16.520488,0.334587
4,Precious_Metals_Spot,26.268666,33.813739,-1.949706


In [26]:
commodity[-12:].std()

Agriculture_Spot          47.925658
Energy_Spot               97.699191
Industrial_Metals_Spot    35.374595
Livestock_Spot            13.416705
Precious_Metals_Spot      22.730990
dtype: float64